In [1]:
import pandas as pd
import numpy as np
import os
import math
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score

In [2]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV


In [3]:
# filepath = "merged_u1-50_w001_resampled.csv"
filepath = "D:/online dataset csv files preprocessing/merged_u1-50_w001_resampled.csv"

df_original = pd.read_csv(filepath)

In [4]:
df_original

,timestamp,LAx,LAy,LAz,GYx,GYy,GYz,name
0,924313410000000,1.242813,-0.160181,-0.900440,0.194931,-0.004242,0.163818,1
1,924313420000000,1.117246,-0.187921,-1.529893,0.236458,0.076378,0.133293,1
2,924313430000000,0.912846,-0.192566,-1.570929,0.226074,0.012253,0.108246,1
3,924313440000000,0.874858,-0.227208,-1.453511,0.139336,-0.050667,0.085030,1
4,924313450000000,0.834022,-0.282976,-1.292671,-0.013374,-0.109314,0.078308,1
...,...,...,...,...,...,...,...,...
2055043,1368327120000000,-9.502644,-7.178468,4.246279,-2.834417,-0.120428,-0.987857,50
2055044,1368327130000000,-9.443804,-7.698220,3.059675,-2.638940,0.561997,-0.947715,50
2055045,1368327140000000,-8.345459,-6.805815,1.853457,-2.483606,1.225222,-0.954696,50
2055046,1368327150000000,-7.580540,-5.629017,1.559257,-2.429500,1.769765,-1.003565,50


In [5]:
df_original = df_original.drop("timestamp", axis = 1)
df_original['name'] = df_original['name'] - 1
df_original

,LAx,LAy,LAz,GYx,GYy,GYz,name
0,1.242813,-0.160181,-0.900440,0.194931,-0.004242,0.163818,0
1,1.117246,-0.187921,-1.529893,0.236458,0.076378,0.133293,0
2,0.912846,-0.192566,-1.570929,0.226074,0.012253,0.108246,0
3,0.874858,-0.227208,-1.453511,0.139336,-0.050667,0.085030,0
4,0.834022,-0.282976,-1.292671,-0.013374,-0.109314,0.078308,0
...,...,...,...,...,...,...,...
2055043,-9.502644,-7.178468,4.246279,-2.834417,-0.120428,-0.987857,49
2055044,-9.443804,-7.698220,3.059675,-2.638940,0.561997,-0.947715,49
2055045,-8.345459,-6.805815,1.853457,-2.483606,1.225222,-0.954696,49
2055046,-7.580540,-5.629017,1.559257,-2.429500,1.769765,-1.003565,49


In [6]:
X_train = pd.DataFrame(columns = ['LAx', 'LAy', 'LAz','GYx', 'GYy', 'GYz','name'] )
X_test = pd.DataFrame(columns = ['LAx', 'LAy', 'LAz','GYx', 'GYy', 'GYz','name'] )
    
for i in range(0,50):
    df = df_original[df_original['name'] == i].copy()
    df_train = df[:-600]
    df_test = df[-600:]
    
    X_train = pd.concat([X_train, df_train])
    X_test = pd.concat([X_test, df_test])


In [7]:
X_train

,LAx,LAy,LAz,GYx,GYy,GYz,name
0,1.242813,-0.160181,-0.900440,0.194931,-0.004242,0.163818,0
1,1.117246,-0.187921,-1.529893,0.236458,0.076378,0.133293,0
2,0.912846,-0.192566,-1.570929,0.226074,0.012253,0.108246,0
3,0.874858,-0.227208,-1.453511,0.139336,-0.050667,0.085030,0
4,0.834022,-0.282976,-1.292671,-0.013374,-0.109314,0.078308,0
...,...,...,...,...,...,...,...
2054443,2.981221,-10.895188,-12.542705,0.076795,0.586431,0.490438,49
2054444,2.451662,-10.944221,-8.610238,-0.900591,0.104720,0.029671,49
2054445,1.117958,-11.003061,-4.383573,-1.130974,-0.593412,-0.452041,49
2054446,0.235360,-6.050703,-2.451662,-1.026255,-1.130974,-0.787144,49


In [8]:
y_train = X_train.pop('name')
y_test = X_test.pop('name')

In [9]:
X_train.shape , X_test.shape, y_train.shape, y_test.shape

((2025048, 6), (30000, 6), (2025048,), (30000,))

In [10]:
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test)
dtrain, dtest

(<xgboost.core.DMatrix at 0x2146f2bbb20>,
 <xgboost.core.DMatrix at 0x2146f2ba8f0>)

In [11]:
params = {
    'max_depth': 7,
    'learning_rate': 0.3,
    'objective': 'multi:softmax',
    'num_class': 50
}
epochs = 15
evals_result = {}

In [ ]:
model = xgb.train(params, dtrain, epochs,evals=[(dtrain, 'train')],evals_result=evals_result,  verbose_eval=1)

[0]	train-mlogloss:2.95717


In [ ]:
predictions = model.predict(dtest)
print(predictions)

In [ ]:
predictions.dtype

In [ ]:
y_test_typed = y_test.astype(np.float32)

In [ ]:
y_test_typed.dtype

In [ ]:
accuracy_score(y_test_typed, predictions)